<h1><span style = "color: blue; font-weight:bold" > Scrapping Covid Data from Ministry of Health</span></h1>

This notebook is to collect the covid situation which is happenning in Vietnam

<h2> 1. Import library</h2>

In [256]:
import pandas as pd
import numpy as np
import requests
import requests
from bs4 import BeautifulSoup
import lxml.html
import datetime
import time
import pandas as pd
import re
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import matplotlib.pyplot as plt
import seaborn as sns
import urllib3

<h2> 2. Scrapped Data </h2>

In [2]:
url = "https://ncov.moh.gov.vn/vi/web/guest/trang-chu?p_p_id=corona_trangchu_top_CoronaTrangchuTopPortlet_INSTANCE_RrVAbIFIPL7v&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_corona_trangchu_top_CoronaTrangchuTopPortlet_INSTANCE_RrVAbIFIPL7v_delta=100&_corona_trangchu_top_CoronaTrangchuTopPortlet_INSTANCE_RrVAbIFIPL7v_resetCur=false&_corona_trangchu_top_CoronaTrangchuTopPortlet_INSTANCE_RrVAbIFIPL7v_cur={}"

In [3]:
def get_soup_data(link, m):
    final_soup = []
    for i in tqdm(range (0,m)):
        page = requests.get(link.format(i), verify = False)
        soup = BeautifulSoup(page.text, 'html.parser')
        soup = soup.find_all('div', attrs= {"class": "inner"}) 
        final_soup.append(soup)
    return final_soup

In [52]:
urllib3.disable_warnings()
soup = get_soup_data(url,120)

100%|██████████| 120/120 [03:40<00:00,  1.83s/it]


<h2> 3. Extract & Clean data </h2>

In [110]:
def clean_tag(text):
    patient_n_age = [m.find_all('span',attrs = {"class":"text-uppercase"}) for i in soup for m in i]
    patient_n_age = [i.get_text() for m in patient_n_age for i in m ]
    loc_sta_nat  = [m.find_all('p') for i in soup for m in i]
    loc_sta_nat = [i.get_text() for m in loc_sta_nat for i in m ]
    return patient_n_age, loc_sta_nat


In [149]:
patient_n_age, loc_sta_nat = clean_tag(soup)

In [173]:
patient_number = [ int(re.sub(r"(\t)|BN|\(|\)|\,", "", i.split(" -")[0])) for i in patient_n_age ]

age = [ int(i.split(" - ")[1].replace("tuổi","").replace(" ","")) for i in patient_n_age ]
province = [ i.split("\n")[1].replace(" I","") for i in loc_sta_nat]
status = [ i.split("\n")[2].replace("I","") for i in loc_sta_nat]
nationality = [ i.split("\n")[3].replace("\r","") for i in loc_sta_nat]

In [177]:
data =pd.DataFrame(list(zip(patient_number,age,province,status,nationality)), 
                  columns = ['patient_number','age','province','status','nationality'])
    

In [178]:

data.head()

,patient_number,age,province,status,nationality
0,10999,42,Hồ Chí Minh,Đang điều trị,Việt Nam
1,10998,70,Hồ Chí Minh,Đang điều trị,Việt Nam
2,10997,41,Hồ Chí Minh,Đang điều trị,Việt Nam
3,10996,16,Hồ Chí Minh,Đang điều trị,Việt Nam
4,10995,43,Hồ Chí Minh,Đang điều trị,Việt Nam


In [179]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11990 entries, 0 to 11989
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   patient_number  11990 non-null  int64 
 1   age             11990 non-null  int64 
 2   province        11990 non-null  object
 3   status          11990 non-null  object
 4   nationality     11990 non-null  object
dtypes: int64(2), object(3)
memory usage: 468.5+ KB


In [180]:
data['status']

0        Đang điều trị
1        Đang điều trị
2        Đang điều trị
3        Đang điều trị
4        Đang điều trị
             ...      
11985             Khỏi
11986             Khỏi
11987             Khỏi
11988             Khỏi
11989             Khỏi
Name: status, Length: 11990, dtype: object

<h2> 4. Translate to English </h2>

In [181]:
import google_trans_new
from google_trans_new import google_translator  

Demo translate

In [182]:
translator = google_translator()  
print(translator.translate('Đang điều trị',lang_tgt='en'))

Being treated 


Loop to translate all rows in Vietnamese to English

In [183]:
translator = google_translator()
translations = {}
for column in data.columns[2:5]:
    # Unique elements of the column
    unique_elements = data[column].unique()
    for element in unique_elements:
        # Adding all the translations to a dictionary (translations)
        translations[element] = translator.translate(element,lang_tgt='en')


In [184]:
import itertools
dict(itertools.islice(translations.items(), 3)) 

{'Hồ Chí Minh': 'Ho Chi Minh ', 'Bắc Giang': 'Bac Giang ', 'Hà Nội': 'Hanoi '}

In [185]:
adj_dic = {'Thái Bình': 'Thai Binh','Hà Nam': 'Ha Nam ','Hoà Bình': 'Hoa Binh ',
           'Gia Lai': 'Gia Lai','Bình Định': 'Binh Dinh','Khỏi': 'Recovered',
           'Áo': 'Austria','Đức': 'Germany', 'U-dơ-bê-ki-xtan': 'Uzbekistan',
           'Ba Lan': 'Poland','Nga': 'Russia','Bra-xin': 'Brazil','Li-bi': 'Libya',
           'Nê-pan': 'Nepal ','Hung-ga-ri': 'Hungary ','Pháp': 'France ','Mi-an-ma': 'Myanmar',
          'Lát-vi-a': 'Latvia', 'Moldova': 'Moldova', 'And Islands ':'Vietnam'}

In [186]:
for i in translations.keys():
    if i in adj_dic.keys():
        translations[i] = adj_dic[i]

In [187]:
data.replace(translations, inplace = True)

In [188]:
data.head()

,patient_number,age,province,status,nationality
0,10999,42,Ho Chi Minh,Being treated,Vietnam
1,10998,70,Ho Chi Minh,Being treated,Vietnam
2,10997,41,Ho Chi Minh,Being treated,Vietnam
3,10996,16,Ho Chi Minh,Being treated,Vietnam
4,10995,43,Ho Chi Minh,Being treated,Vietnam


<h2> 5. Basic EDA </h2>

This is to explore answers for these questions:
1. Number of cases by province and country
2. Number of cases by status


<b>1. Number of cases by province and country</b>

In [350]:
data.groupby('province').patient_number.count().sort_values(ascending = False)

province
Bac Giang           4317
Bac Ninh            1417
Ho Chi Minh         1248
Hai Duong            812
Hanoi                732
Danang               654
Khanh Hoa            145
BA Ria Vung Tau      138
Quang Nam            127
Vinh Phuc            111
Lang Son             101
Quang Ninh            90
Ha Tinh               77
hung Yen              75
Tay Ninh              65
Thai Binh             63
Kien Giang            63
Dien Bien             59
Binh Duong            58
Ha Nam                55
Bac Lieu              53
Ninh Binh             40
Tien Giang            35
Can Tho               34
Hoa Binh              33
Dong Nai              31
Dong Thap             29
Gia Lai               28
Thanh Hoa             28
Phu Yen               28
Long An               26
Nam Dinh              25
An Giang              21
Ben tre               20
Soc Trang             19
Vinh Long             15
Quang Tri             13
Ninh Thuan            12
Hai Phong             12
Binh Thuan      

In [351]:
data.groupby('nationality').patient_number.count().sort_values(ascending = False)

nationality
Vietnam                         10762
India                              60
Russia                             28
UK                                 22
China                              14
USA                                12
Philippines                        11
South Africa                        8
France                              7
Brazil                              7
Japan                               6
Germany                             6
Taiwan                              5
Korea                               4
Austria                             4
Belarus                             3
Turkey                              3
Costa Rica Republic                 3
Ukraine                             2
Israel                              2
And Islands                         2
Myanmar                             2
Pakistan                            2
Australia                           1
Uzbekistan                          1
Poland                              1


<b> 2. Number of cases by status </b>

In [352]:
data.groupby('status').patient_number.count()

status
Being treated     6700
Dead                59
Other                4
Recovered         4236
Name: patient_number, dtype: int64

In [353]:
data[data.status == 'Other ']

,patient_number,age,province,status,nationality
10681,418,61,Danang,Other,Vietnam
10646,453,56,Danang,Other,Vietnam
10335,764,67,Danang,Other,Vietnam
8870,2229,54,Hanoi,Other,Japan


Status "Other" are used to refer to 4 patients that die because of other reason not covid
+ Patient 418: died because of chronic kidney disease ( 4 times negative test covid)
+ Patient 453: died because of chronic kidney disease, pneumonia, heart failure, stroke ( 3 times negative test covid)
+ Patient 764: died because of chronic kidney disease,hypertension, stroke ( 3 times negative test covid)
+ Patient 2229: died unknown reason ( 2 times negative test covid)

<b> 3. Total cases </b>

In [354]:
data

,patient_number,age,province,status,nationality
11098,1,66,Ho Chi Minh,Recovered,China
11097,2,28,Ho Chi Minh,Recovered,China
11096,3,25,Thanh Hoa,Recovered,Vietnam
11095,4,29,Vinh Phuc,Recovered,Vietnam
11094,5,23,Vinh Phuc,Recovered,Vietnam
...,...,...,...,...,...
4,10995,43,Ho Chi Minh,Being treated,Vietnam
3,10996,16,Ho Chi Minh,Being treated,Vietnam
2,10997,41,Ho Chi Minh,Being treated,Vietnam
1,10998,70,Ho Chi Minh,Being treated,Vietnam


In [191]:
#data.to_csv("data15jun.csv")

In [278]:
#data = pd.read_csv("data15jun.csv")

In [303]:
#dailycase(dc)
gitlink = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"

dc = pd.read_csv(gitlink)


In [304]:
dc =dc[dc['Country/Region']=='Vietnam']
dc.rename(index= str, columns = {"Country/Region": "Country"}, inplace = True)
dc.drop(columns = ['Country','Lat', 'Long','Province/State'], axis = 1, inplace = True)

In [305]:
dc = dc.melt( 
        var_name="Date", 
        value_name="Cases")

In [306]:
dc

,Date,Cases
0,1/22/20,0
1,1/23/20,2
2,1/24/20,2
3,1/25/20,2
4,1/26/20,2
...,...,...
505,6/10/21,9835
506,6/11/21,10048
507,6/12/21,10337
508,6/13/21,10630


In [307]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11990 entries, 0 to 11989
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      11990 non-null  int64 
 1   patient_number  11990 non-null  int64 
 2   age             11990 non-null  int64 
 3   province        11988 non-null  object
 4   status          11990 non-null  object
 5   nationality     11989 non-null  object
dtypes: int64(3), object(3)
memory usage: 562.2+ KB


In [308]:
data.drop(['Unnamed: 0'], axis = 1, inplace = True)
data.drop_duplicates(inplace = True)

In [309]:
data.sort_values('patient_number', ascending = True).head(100)

,patient_number,age,province,status,nationality
11098,1,66,Ho Chi Minh,Recovered,China
11097,2,28,Ho Chi Minh,Recovered,China
11096,3,25,Thanh Hoa,Recovered,Vietnam
11095,4,29,Vinh Phuc,Recovered,Vietnam
11094,5,23,Vinh Phuc,Recovered,Vietnam
...,...,...,...,...,...
11003,96,21,Ho Chi Minh,Recovered,Vietnam
11002,97,34,Ho Chi Minh,Recovered,UK
11001,98,34,Ho Chi Minh,Recovered,UK
11000,99,29,Ho Chi Minh,Recovered,Vietnam


In [312]:
dc.drop_duplicates(subset = ['Cases'] ,keep = 'first', inplace= True)

In [ ]:
data.loc[data.patient_number == 92859285,'patient_number']=9285


In [341]:
data = data.sort_values('patient_number')

In [338]:
data.head()

,patient_number,age,province,status,nationality
11098,1,66,Ho Chi Minh,Recovered,China
11097,2,28,Ho Chi Minh,Recovered,China
11096,3,25,Thanh Hoa,Recovered,Vietnam
11095,4,29,Vinh Phuc,Recovered,Vietnam
11094,5,23,Vinh Phuc,Recovered,Vietnam


In [315]:
dc.drop([0], inplace = True)

In [316]:
dc.head()

,Date,Cases
1,1/23/20,2
10,2/1/20,6
12,2/3/20,8
15,2/6/20,10
17,2/8/20,13


In [342]:
df =pd.merge_asof(data, dc, direction="forward", left_on='patient_number', right_on='Cases' )

In [343]:
df.head()

,patient_number,age,province,status,nationality,Date,Cases
0,1,66,Ho Chi Minh,Recovered,China,1/23/20,2.0
1,2,28,Ho Chi Minh,Recovered,China,1/23/20,2.0
2,3,25,Thanh Hoa,Recovered,Vietnam,2/1/20,6.0
3,4,29,Vinh Phuc,Recovered,Vietnam,2/1/20,6.0
4,5,23,Vinh Phuc,Recovered,Vietnam,2/1/20,6.0


In [346]:
df.loc[df.Cases.isna(),'Date']='6/15/21'

In [347]:
df.loc[df.Cases.isna(),'Cases']=10999

,patient_number,age,province,status,nationality,Date,Cases
10849,10850,28,Bac Giang,Being treated,Vietnam,6/14/21,10881.0
10850,10851,34,Bac Giang,Being treated,Vietnam,6/14/21,10881.0
10851,10852,19,Bac Giang,Being treated,Vietnam,6/14/21,10881.0
10852,10853,32,Bac Giang,Being treated,Vietnam,6/14/21,10881.0
10853,10854,28,Bac Giang,Being treated,Vietnam,6/14/21,10881.0
...,...,...,...,...,...,...,...
10994,10995,43,Ho Chi Minh,Being treated,Vietnam,6/15/21,10999.0
10995,10996,16,Ho Chi Minh,Being treated,Vietnam,6/15/21,10999.0
10996,10997,41,Ho Chi Minh,Being treated,Vietnam,6/15/21,10999.0
10997,10998,70,Ho Chi Minh,Being treated,Vietnam,6/15/21,10999.0
